In [2]:
import pandas as pd
import matplotlib.pyplot as plt

## Cleaning Cannabis Arrest Data

In [3]:
# Load in cannabis arrest data and clean

COL_NAMES_ARREST = {
    'African American': 'blackArrests',
    'Asian': 'asianArrests',
    'Hispanic': 'hispanicArrests',
    'Native American': 'naAmArrests',
    'Native Hawaiian / Pacific Islander': 'naHiPaIsArrests',
    'Unknown': 'unknownArrests',
    'White': 'whiteArrests'
}

marijuana_arrests = pd.read_csv("../data/Marijuana_Arrests.csv")

marijuana_arrests.columns = [x.strip() for x in marijuana_arrests.columns]
marijuana_arrests["County"] = marijuana_arrests["County"].replace(
    to_replace="#NULL", value=None
)

marijuana_arrests["RaceCode"] = marijuana_arrests["RaceCode"].replace(
    to_replace="Native Hawaiian / Pacific Ilandar",
    value="Native Hawaiian / Pacific Islander")
marijuana_arrests["RaceCode"] = marijuana_arrests["RaceCode"].replace(
    to_replace="Hispianic",
    value="Hispanic")

# manipulate dataframe to get arrest totals by county-year with racial composition
marijuana_arrests = marijuana_arrests.groupby(
    ['County', 'ArrestYear', 'RaceCode']
).sum().reset_index()

marijuana_arrests = marijuana_arrests.pivot(
    index=['County', 'ArrestYear'],
    columns='RaceCode',
    values='NumberOfArrests'
).reset_index().fillna(0)

marijuana_arrests.rename(columns=COL_NAMES_ARREST, inplace=True)

marijuana_arrests['totalArrests'] = marijuana_arrests.blackArrests + \
    marijuana_arrests.asianArrests + marijuana_arrests.hispanicArrests + \
    marijuana_arrests.naAmArrests + marijuana_arrests.naHiPaIsArrests + \
    marijuana_arrests.unknownArrests + marijuana_arrests.whiteArrests

marijuana_arrests['blackPct'] = marijuana_arrests.blackArrests / marijuana_arrests.totalArrests

marijuana_arrests['countyWeight'] = marijuana_arrests.totalArrests / marijuana_arrests.totalArrests.sum()

In [15]:
count = marijuana_arrests.groupby('County').size().reset_index().rename(columns={0: 'cnt'})
mask = count.cnt == 6
count[mask].County.values

array(['Adams', 'Alamosa', 'Arapahoe', 'Archuleta', 'Boulder',
       'Broomfield', 'Chaffee', 'Clear Creek', 'Custer', 'Delta',
       'Denver', 'Douglas', 'Eagle', 'El Paso', 'Elbert', 'Fremont',
       'Garfield', 'Gilpin', 'Gunnison', 'Jefferson', 'Kit Carson',
       'La Plata', 'Larimer', 'Logan', 'Mesa', 'Moffat', 'Montezuma',
       'Montrose', 'Morgan', 'Otero', 'Prowers', 'Pueblo', 'Rio Blanco',
       'Rio Grande', 'Routt', 'Summit', 'Teller', 'Weld'], dtype=object)

In [13]:
marijuana_arrests.to_json('../data/cannabis_arrests.json', orient='records')

## Cleaning ACS 5-Year Estimates (2017) for CO County Population Data by Race

In [10]:
# Load and clean county population data from ACS 5-Year Estimates (2017)

COL_NAMES = {
    'B03002_001E': 'totalPop',
    'B03002_003E': 'whitePop',
    'B03002_004E': 'blackPop',
    'B03002_005E': 'amInAlNaPop',
    'B03002_006E': 'asianPop',
    'B03002_007E': 'naHiPaIsPop',
    'B03002_008E': 'otherPop',
    'B03002_012E': 'hispanicPop'
}

acs17 = pd.read_csv(
    "../data/acs5YR2017_co_counties.csv", 
    skiprows=[1]
)

acs17["County"] = acs17["NAME"].map(
    lambda x: x.replace(" County, Colorado", "")
)

acs17['twoOrMoreRacePop'] = acs17.B03002_009E

acs17.rename(columns=COL_NAMES, inplace=True)

acs17.drop(
    labels=(["GEO_ID", "NAME"] + [x for x in acs17.columns if x[-1] == "M"]), 
    axis=1,
    inplace=True
)

acs17.drop(
    labels=([x for x in acs17.columns if x[0] == "B"]), 
    axis=1,
    inplace=True
)

# acs17['totalRacePop'] = acs17.whitePop + acs17.blackPop + acs17.hispanicPop + acs17.amInAlNaPop + acs17.asianPop + acs17.naHiPaIsPop + acs17.twoOrMoreRacePop + acs17.otherPop

state_totals = acs17.sum(axis=0)
new_row = pd.DataFrame(state_totals.values.reshape(1, 10), columns=state_totals.index)
acs17 = acs17.append(new_row, ignore_index=True)
# acs17.iloc[64, 0] = 'statewide'

acs17['blackPct'] = acs17.blackPop / acs17.totalPop

acs17 = acs17[['County', 'totalPop', 'whitePop', 'blackPop', 'blackPct', 'hispanicPop', 'amInAlNaPop', 'asianPop', 'naHiPaIsPop', 'twoOrMoreRacePop', 'otherPop']]

acs17.iloc[64, 0] = 'statewide'

In [11]:
acs17

,County,totalPop,whitePop,blackPop,blackPct,hispanicPop,amInAlNaPop,asianPop,naHiPaIsPop,twoOrMoreRacePop,otherPop
0,Adams,487850,249239,14662,0.030054,191857,2382,18605,604,9543,958
1,Alamosa,16345,7948,264,0.016152,7488,206,168,26,207,38
2,Arapahoe,626612,384195,63646,0.101572,118350,1937,35322,1286,20179,1697
3,Archuleta,12592,9661,136,0.010801,2309,267,92,0,102,25
4,Baca,3581,3069,52,0.014521,376,39,0,0,45,0
...,...,...,...,...,...,...,...,...,...,...,...
60,Teller,23769,21266,158,0.006647,1469,91,241,0,536,8
61,Washington,4809,4220,45,0.009357,460,13,22,0,49,0
62,Weld,285729,189741,2956,0.010345,82920,1129,3754,250,4590,389
63,Yuma,10109,7649,46,0.00455,2289,27,0,0,95,3


In [15]:
acs17.to_json('../data/coPopulation.json', orient='records')

## Merge Arrest Data with Population Data and Save as JSON

In [103]:
import json
merged = pd.merge(
    marijuana_arrests,
    acs17,
    on='County'
)
merged

,County,ArrestYear,blackArrests,asianArrests,hispanicArrests,naAmArrests,naHiPaIsArrests,unknownArrests,whiteArrests,totalArrests,totalPop,whitePop,blackPop,hispanicPop,amInAlNaPop,asianPop,naHiPaIsPop,twoOrMoreRacePop,otherPop
0,Adams,2012,96.0,17.0,859.0,6.0,0.0,12.0,1317.0,2307.0,487850,249239,14662,191857,2382,18605,604,19086,958
1,Adams,2013,39.0,11.0,417.0,3.0,0.0,7.0,515.0,992.0,487850,249239,14662,191857,2382,18605,604,19086,958
2,Adams,2014,29.0,2.0,353.0,0.0,0.0,5.0,458.0,847.0,487850,249239,14662,191857,2382,18605,604,19086,958
3,Adams,2015,38.0,12.0,348.0,1.0,0.0,3.0,386.0,788.0,487850,249239,14662,191857,2382,18605,604,19086,958
4,Adams,2016,36.0,7.0,326.0,4.0,0.0,5.0,336.0,714.0,487850,249239,14662,191857,2382,18605,604,19086,958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,Weld,2016,21.0,4.0,84.0,2.0,0.0,0.0,428.0,539.0,285729,189741,2956,82920,1129,3754,250,9180,389
298,Weld,2017,33.0,3.0,85.0,1.0,0.0,2.0,299.0,423.0,285729,189741,2956,82920,1129,3754,250,9180,389
299,Yuma,2012,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,10109,7649,46,2289,27,0,0,190,3
300,Yuma,2013,6.0,0.0,36.0,1.0,0.0,0.0,120.0,163.0,10109,7649,46,2289,27,0,0,190,3


In [106]:
result = merged.to_json('../data/arrests.json', orient='records')